In [ ]:
import sims
import numpy as np
import msprime
import pandas as pd

import useful
import seaborn
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload


In [ ]:
# Generation time, mutation rate and recomination rate
RR = 1e-8
MU = 1.29e-8 
GEN_time = 29.0 

# Split Times
T_NEAND_migration = 55000 #time of Neanderthal migration into Out_of_africa population
T_NEAND_AMH = 550000 # split time between AMH and Neanderthal
T_OOF_AF = 65700 # Out_of_Africa migration time
T_NEAND_samples = 38000

# Effective population size
N_ANC = 18500 # N_e of common  AMH and NEanderthal population 
N_ND = 3400 # N_e of Neanderthal
N_AMH = 23000 # N_e of AMH
N_OOF = 1861 # N_e of Out_of_Africa population
N_AF = 27600 # N_e of Africans
N_EU = 13377 #N_e of Europeans

N_EU_bottleneck = 1080
N_EU_growth = 1450
T_EU_growth = 31900
gr_rate = 0.00202
Portion_admix = 0.03

len_sequence = 1e7 # DNA sequence length

n = 500 # number of generated   AF samples
n_neand = 10 #number of generated Neanderthals

rand_sd =1234 #random seed



T = np.array([T_NEAND_migration, T_NEAND_AMH, T_OOF_AF])/GEN_time

In [ ]:
N_ND = 3400 # N_e of Neanderthal
N_e = np.array([N_ANC, N_ND, N_AMH, N_OOF, N_AF, N_EU])

n_eu=1
ploidy=2
ts =sims.history_archaic(GEN_time, len_sequence, RR, MU, N_e, T,  n, rand_sd, n_neand,  
                          T_NEAND_samples/GEN_time, n_eu, N_EU_growth, 
                          T_EU_growth/GEN_time, N_EU_bottleneck, gr_rate, Portion_admix, ploidy)
sims.print_neand_dosages(ts)

In [ ]:
ND_true_tracts=sims.real_nd_tracts(ts, n_eu, ploidy,T)
#np.save('ND_true_tracts.npy', np.array(ND_true_tracts, dtype= object))
#np.load("ND_true_tracts.npy", allow_pickle= True)

In [ ]:
N_ref_pop=100 #reference African haplotypes 
N_neanderthal=6
L=1000 #basic window length
N=2 #number of HMM hidden states
sims.create_obs_txt('obs.txt', ploidy, n_eu, ts, N_ref_pop, N_neanderthal, n) #'obs.txt' file with observations
sims.create_bed_smpls_arch_cov('samples.txt', 'test.bed', 'arch.cover.txt', len_sequence, 0.999, 1, n_eu, ploidy) 
#samples.txt file with haplotype names,
#test.bed bed file 

In [ ]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM no --prepared_file obs.txt --o out --arch_cover arch.cover.txt --decoding posterior --cut_off 0.8

In [ ]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM yes --EM_samples 1 --EM_steps 10 --prepared_file obs.txt --o out.EM --arch_cover arch.cover.txt --decoding posterior --cut_off 0.8

In [ ]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM no --prepared_file obs.txt --o out.viterbi --arch_cover arch.cover.txt --decoding viterbi

In [ ]:

ND_HMM_tracts=useful.read_out('out.archaic.txt')

HMM_tracts=[]
REAL_tracts=[]
for idx in range(ploidy*n_eu):
    HMM_tracts.append([useful.tracts_eu(ND_HMM_tracts[idx], len_sequence ), ND_HMM_tracts[idx]])
    REAL_tracts.append([ useful.tracts_eu(ND_true_tracts[idx], len_sequence), ND_true_tracts[idx]])



df=sims.df_result(REAL_tracts, HMM_tracts, N_neanderthal, L, N_ref_pop, n_eu, N, ploidy)
df